In [1]:
import sys

sys.path.append("./optimisation_service")
sys.path.append("./epoch_simulator_lib/build/epoch_py/")

sys.path[0], sys.path[-1] = sys.path[-1], sys.path[0]

In [2]:
from app.models.epoch_types.site_range_type import Building, Config, GasHeater, GasTypeEnum, Grid, SolarPanel, SiteRange, HeatPump, HeatSourceEnum, DomesticHotWater, BatteryModeEnum, EnergyStorageSystem
import numpy as np

site_ranges = {}

building = Building(
    COMPONENT_IS_MANDATORY=True,
    scalar_heat_load=[1],
    scalar_electrical_load=[1],
    fabric_intervention_index=list(range(64)),
    incumbent=False,
    age=0,
)
config = Config(capex_limit=99999999999, use_boiler_upgrade_scheme=False, general_grant_funding=0.0)
grid = Grid(
    COMPONENT_IS_MANDATORY=True,
    import_headroom=[0.4],
    grid_export=[999],
    grid_import=[999],
    tariff_index=[0, 1, 3],
    export_tariff=[0.15],
    incumbent=True,
    age=0,
)
heat_pump = HeatPump(
    COMPONENT_IS_MANDATORY = False,
    heat_power = list(range(50))[1:],
    heat_source = [HeatSourceEnum.AMBIENT_AIR],
    send_temp = [2.0],
    incumbent = False,
    age = 0,
    lifetime = 10
)
gas_heater = GasHeater(
    COMPONENT_IS_MANDATORY=False,
    maximum_output=[999999],
    gas_type=[GasTypeEnum.NATURAL_GAS],
    boiler_efficiency=[0.95],
    incumbent=True,
    age=0,
)
domestic_hot_water = DomesticHotWater(
    COMPONENT_IS_MANDATORY = False,
    cylinder_volume = list(range(200))[1:],
    incumbent = False,
    age = 0,
    lifetime = 12,
)
energy_storage_system = EnergyStorageSystem(
    COMPONENT_IS_MANDATORY = False,
    capacity = list(range(100))[1:],
    charge_power = list(range(50))[1:],
    discharge_power = list(range(50))[1:],
    battery_mode = [BatteryModeEnum.CONSUME, BatteryModeEnum.CONSUME_PLUS],
    initial_charge = [0.0],
    incumbent = False,
    age = 0,
    lifetime = 15,
)

In [ ]:
solar_panels = [
    SolarPanel(
        COMPONENT_IS_MANDATORY=False, yield_scalar=np.arange(0.5, 52.5, 0.5).tolist(), yield_index=[0], incumbent=False, age=0, lifetime=25
    ),
    SolarPanel(
        COMPONENT_IS_MANDATORY=False, yield_scalar=np.arange(0.5, 79.5, 0.5).tolist(), yield_index=[0], incumbent=False, age=0, lifetime=25
    )
]

site_ranges["A"] = SiteRange(building=building, grid=grid, gas_heater=gas_heater, solar_panels=solar_panels, heat_pump=heat_pump, domestic_hot_water=domestic_hot_water, energy_storage_system=energy_storage_system)

In [ ]:
solar_panels = [
    SolarPanel(
        COMPONENT_IS_MANDATORY=False, yield_scalar=np.arange(0.5, 8.0, 0.5).tolist(), yield_index=[0], incumbent=False, age=0, lifetime=25
    ),
    SolarPanel(
        COMPONENT_IS_MANDATORY=False, yield_scalar=np.arange(0.5, 3.0, 0.5).tolist(), yield_index=[0], incumbent=False, age=0, lifetime=25
    ),
    SolarPanel(
        COMPONENT_IS_MANDATORY=False, yield_scalar=np.arange(0.5, 6.0, 0.5).tolist(), yield_index=[0], incumbent=False, age=0, lifetime=25
    )
]

site_ranges["B"] = SiteRange(building=building, grid=grid, gas_heater=gas_heater, solar_panels=solar_panels, heat_pump=heat_pump, domestic_hot_water=domestic_hot_water, energy_storage_system=energy_storage_system)

In [ ]:
solar_panels = [
    SolarPanel(
        COMPONENT_IS_MANDATORY=False, yield_scalar=np.arange(0.5, 5.5, 0.5).tolist(), yield_index=[0], incumbent=False, age=0, lifetime=25
    ),
    SolarPanel(
        COMPONENT_IS_MANDATORY=False, yield_scalar=np.arange(0.5, 9.5, 0.5).tolist(), yield_index=[0], incumbent=False, age=0, lifetime=25
    ),
    SolarPanel(
        COMPONENT_IS_MANDATORY=False, yield_scalar=np.arange(0.5, 5.5, 0.5).tolist(), yield_index=[0], incumbent=False, age=0, lifetime=25
    ),
    SolarPanel(
        COMPONENT_IS_MANDATORY=False, yield_scalar=np.arange(0.5, 5.0, 0.5).tolist(), yield_index=[0], incumbent=False, age=0, lifetime=25
    )
]

site_ranges["C"] = SiteRange(building=building, grid=grid, gas_heater=gas_heater, solar_panels=solar_panels, heat_pump=heat_pump, domestic_hot_water=domestic_hot_water, energy_storage_system=energy_storage_system)

In [ ]:
import datetime
from pathlib import Path
from app.models.core import Site
from app.models.site_data import LegacySiteMetaData
from tests.conftest import load_epoch_data_from_file
from app.models.epoch_types.config import Config
from app.internal.constraints import apply_default_constraints
from app.models.constraints import Bounds
from app.models.metrics import Metric

site_ids = ["A", "B", "C"]
portfolio = []
for site_id in site_ids:
    start_ts = datetime.datetime(year=2022, month=1, day=1, hour=0, tzinfo=datetime.UTC)
    end_ts = datetime.datetime(year=2023, month=1, day=1, hour=0, tzinfo=datetime.UTC)
    site_data = LegacySiteMetaData(site_id=site_id, start_ts=start_ts, end_ts=end_ts)

    site = Site(name=site_id, site_range=site_ranges[site_id], site_data=site_data, constraints={}, config=Config())

    epoch_data = load_epoch_data_from_file(path=Path("bo_test_data", f"{site_id}.json"))

    site._epoch_data = epoch_data

    portfolio.append(site)

constraints = {Metric.capex: Bounds(max=100000)}

portfolio, constraints = apply_default_constraints(portfolio, constraints)

objectives = [Metric.npv_balance, Metric.carbon_balance_total]

In [7]:
from app.internal.NSGA2 import NSGA2

In [8]:
nsga = NSGA2(pop_size=512, n_offsprings=256, period=5)

nsga_res = nsga.run(objectives=objectives, constraints=constraints, portfolio=portfolio)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      512 |      1 |  1.614830E+05 |  1.232251E+06 |             - |             -
     2 |      768 |      1 |  1.614830E+05 |  8.768632E+05 |             - |             -
     3 |     1024 |      1 |  1.291728E+05 |  6.437366E+05 |             - |             -
     4 |     1280 |      1 |  1.291728E+05 |  4.806465E+05 |             - |             -
     5 |     1536 |      1 |  1.267008E+05 |  3.715820E+05 |             - |             -
     6 |     1792 |      1 |  3.616309E+04 |  2.882462E+05 |             - |             -
     7 |     2048 |      1 |  0.000000E+00 |  2.249763E+05 |             - |             -
     8 |     2304 |      1 |  0.000000E+00 |  1.860158E+05 |  3.420919E+04 |         ideal
     9 |     2560 |      2 |  0.000000E+00 |  1.530181E+05 |  1.0000000000 |         ideal
    10 |     2816 |      3 |  0.000000E+00 |  1.222893E+05 |  0.9938397223 |         ideal

In [9]:
from app.internal.bayesian.bayesian import Bayesian as Old_Bayesian

/home/will/git_repos/EPOCH/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from app.models.optimisers import NSGA2HyperParam

NSGA2_param = NSGA2HyperParam(period=5)

In [16]:
old_bo = Old_Bayesian(n_initialisation_points = 5, batch_size = 1, n_generations=10, NSGA2_param=NSGA2_param)

old_bo_res = old_bo.run(objectives=objectives, constraints=constraints, portfolio=portfolio)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      256 |      3 |  0.000000E+00 |  5.641459E+05 |             - |             -
     2 |      384 |      4 |  0.000000E+00 |  2.894353E+05 |  0.0586611563 |         ideal
     3 |      512 |      2 |  0.000000E+00 |  1.472419E+05 |  0.0637980582 |         ideal
     4 |      640 |      2 |  0.000000E+00 |  7.336524E+04 |  0.6538300604 |         ideal
     5 |      768 |      4 |  0.000000E+00 |  2.963965E+04 |  0.0225247411 |         ideal
     6 |      896 |      4 |  0.000000E+00 |  5.004300E+03 |  0.2371385308 |         ideal
     7 |     1024 |      4 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     8 |     1152 |      4 |  0.000000E+00 |  0.000000E+00 |  0.0070608435 |         ideal
     9 |     1280 |      5 |  0.000000E+00 |  0.000000E+00 |  0.0616352865 |         ideal
    10 |     1408 |      7 |  0.000000E+00 |  0.000000E+00 |  0.0503247866 |         ideal

In [17]:
len(old_bo_res.solutions)

52

In [12]:
from app.internal.bayesian.simplified_bo import Bayesian

In [13]:
from app.models.optimisers import NSGA2HyperParam

NSGA2_param = NSGA2HyperParam(period=5)

In [14]:
bo = Bayesian(n_initialisation_points = 5, batch_size = 1, n_generations=10, NSGA2_param=NSGA2_param)

res = bo.run(objectives=objectives, constraints=constraints, portfolio=portfolio)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      256 |      1 |  0.000000E+00 |  5.561052E+05 |             - |             -
     2 |      384 |      1 |  0.000000E+00 |  2.928045E+05 |  0.000000E+00 |             f
     3 |      512 |      1 |  0.000000E+00 |  1.758360E+05 |  0.000000E+00 |             f
     4 |      640 |      1 |  0.000000E+00 |  9.800854E+04 |  0.000000E+00 |             f
     5 |      768 |      1 |  0.000000E+00 |  4.319416E+04 |  0.000000E+00 |             f
     6 |      896 |      1 |  0.000000E+00 |  1.188157E+04 |  0.000000E+00 |             f
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      256 |      1 |  0.000000E+00 |  5.516059E+05 |             - |             -
     2 |      384 |      1 |  0.000000E+00 |  2.748388E+05 |  6.416685E+03 |         ideal
     3 |      512 |      1 |  0.000000E+00 |  1.568512E+05 |  0.000000E+00 |             f

In [15]:
len(res.solutions)

5